In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
from torch.autograd import Variable
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook
import pickle

In [2]:
with open('screen_info.txt','rb') as fl:
    t = pickle.load(fl)
fnames = t[0]
totf = t[1]
binf = t[2]
runfile = 0
fname = fnames[runfile]
bf = binf[runfile]

In [3]:
path = os.getcwd() + '/bioassay-datasets/'
p_fingerprints = []
c_fingerprints = []
labels = []
with open(path+fname+'red_train.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints.append(row[:bf])
        c_fingerprints.append(row[bf:-1])
        labels.append(row[-1])

In [4]:
p_fingerprints = np.asarray(p_fingerprints)[1:]
p_fingerprints = p_fingerprints.astype(int)
c_fingerprints = np.asarray(c_fingerprints)[1:]
c_fingerprints = c_fingerprints.astype(float)

c_fingerprints = (c_fingerprints - np.mean(c_fingerprints,axis=0))/np.std(c_fingerprints,axis=0)

fingerprints = np.concatenate((p_fingerprints,c_fingerprints),axis=1)
#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = p_fingerprints.shape
ip_dim2 = c_fingerprints.shape[1]
labels = labels[1:]
print(no_examples,ip_dim)
print(ip_dim2)
#p_fingerprints[(p_fingerprints==0)] = -1

(3423, 112)
32


In [5]:
#p_fingerprints[(p_fingerprints==0)] = -1

labels2 = np.zeros((len(labels),))
for i,l in enumerate(labels):
    if l=='Active':
        labels2[i] = 1
    else:
        labels2[i] = 0
labels2 = np.asarray(labels2,dtype=np.int)

In [6]:
labels2.dtype

dtype('int64')

In [7]:
from sklearn import svm
from sklearn import metrics

In [8]:
#print((labels2[50:70]))
print(p_fingerprints.shape)

(3423, 112)


In [9]:
# tmp = []
# for w in range(100,110):
#     clf = svm.SVC(kernel = 'rbf',class_weight={1:w})
#     clf.fit(p_fingerprints,labels2)
#     op = clf.predict(p_fingerprints)
#     cm = metrics.confusion_matrix(labels2,op)
#     tmp.append([w,cm.ravel()])
#     print('tn, fp, fn, tp',cm.ravel())

In [10]:
path = os.getcwd() + '/bioassay-datasets/'
p_fingerprints_test = []
c_fp = []
labels = []
with open(path+'AID362red_test.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints_test.append(row[:bf])
        c_fp.append(row[bf:-1])
        labels.append(row[-1])

In [11]:
p_fingerprints_test = np.asarray(p_fingerprints_test)[1:]
p_fingerprints_test = p_fingerprints_test.astype(int)
c_fp = np.asarray(c_fp)[1:]
c_fp = c_fp.astype(float)

c_fp = (c_fp - np.mean(c_fp,axis=0))/np.std(c_fp,axis=0)

fingerprints_test = np.concatenate((p_fingerprints_test,c_fp),axis=1)
#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = p_fingerprints_test.shape
labels = labels[1:]
print(no_examples,ip_dim)
print("total no of 1s",np.sum(p_fingerprints_test))
print("total no of 0s",no_examples*ip_dim-np.sum(p_fingerprints_test))

(856, 112)
('total no of 1s', 6305)
('total no of 0s', 89567)


In [12]:
#p_fingerprints[(p_fingerprints==0)] = -1

labels2_test = np.zeros((len(labels),1))
for i,l in enumerate(labels):
    if l=='Active':
        labels2_test[i] = 1
    else:
        labels2_test[i] = 0
labels2_test = np.asarray(labels2_test,dtype=np.int)

In [26]:
clf1 = svm.SVC(kernel = 'rbf',gamma=0.01,class_weight={1:65})
clf1.fit(p_fingerprints,labels2)
op1 = clf1.predict(p_fingerprints_test)
cm1 = metrics.confusion_matrix(labels2_test,op1)
print('tn, fp, fn, tp',cm1.ravel())

('tn, fp, fn, tp', array([745,  99,   4,   8]))


In [24]:
clf2 = svm.SVC(kernel = 'rbf',gamma=0.004,class_weight={1:69})
clf2.fit(fingerprints,labels2)
op2 = clf2.predict(fingerprints_test)
cm2 = metrics.confusion_matrix(labels2_test,op2)
print('tn, fp, fn, tp',cm2.ravel())

('tn, fp, fn, tp', array([751,  93,   3,   9]))


In [27]:
clf3 = svm.SVC(kernel = 'rbf',gamma = 0.01,class_weight={1:40})
clf3.fit(c_fingerprints,labels2)
op3 = clf3.predict(c_fp)
cm3 = metrics.confusion_matrix(labels2_test,op3)
print('tn, fp, fn, tp',cm3.ravel())

('tn, fp, fn, tp', array([767,  77,   3,   9]))


In [28]:
tmp = op1 + op2 + op3
tmp2 = np.zeros(tmp.shape)
tmp2[tmp>0] = 1
cm4 = metrics.confusion_matrix(labels2_test,tmp2)
print(cm4.ravel())

[662 182   3   9]


In [ ]:
clf2 = svm.SVC(kernel = 'rbf',gamma=1e-3,class_weight={1:70})
clf2.fit(fingerprints,labels2)
op2 = clf2.predict(fingerprints_test)
cm2 = metrics.confusion_matrix(labels2_test,op2)

print('tn, fp, fn, tp',cm2.ravel())